In [1]:
from pathlib import Path
import pandas as pd

silver_path = Path("/content/drive/MyDrive/PROJETOS/SUPPLY CHAIN/ETL/DATA/SILVER")
gold_path = Path("/content/drive/MyDrive/PROJETOS/SUPPLY CHAIN/ETL/DATA/GOLD")
gold_path.mkdir(parents=True, exist_ok=True)

# ler silver
df_produtos_silver = pd.read_parquet(silver_path / "produtos.parquet")
df_lojas_silver = pd.read_parquet(silver_path / "lojas.parquet")
df_calendario_silver = pd.read_parquet(silver_path / "calendario.parquet")

# montar dimensões (pode só copiar por enquanto)
dim_produto = df_produtos_silver.copy()
dim_loja = df_lojas_silver.copy()
dim_tempo = df_calendario_silver.copy()

# salvar gold
dim_produto.to_parquet(gold_path / "dim_produto.parquet", index=False)
dim_loja.to_parquet(gold_path / "dim_loja.parquet", index=False)
dim_tempo.to_parquet(gold_path / "dim_tempo.parquet", index=False)

print("Dimensões GOLD salvas em", gold_path.resolve())


Dimensões GOLD salvas em /content/drive/MyDrive/PROJETOS/SUPPLY CHAIN/ETL/DATA/GOLD


In [2]:
# FATO VENDAS

# LER SILVER
df_vendas_silver = pd.read_parquet(silver_path / "vendas.parquet")

# realizar merge com calendario
fato_vendas = df_vendas_silver.merge(
    df_calendario_silver[['data', 'id_data']],
    on='data',
    how='left'
)

# Remover a coluna 'data' e organizar colunas na ordem do modelo estrela
fato_vendas = fato_vendas[['id_venda', 'id_produto', 'id_loja', 'id_data', 'quantidade', 'valor_total']]

# checar se há algum id_data nulo
print("Registros com id_data nulo:", fato_vendas['id_data'].isna().sum())

# Salvar fato_vendas no GOLD
fato_vendas.to_parquet(gold_path / "fato_vendas.parquet", index=False)

print("fato_vendas salvo em", gold_path / "fato_vendas.parquet")

Registros com id_data nulo: 0
fato_vendas salvo em /content/drive/MyDrive/PROJETOS/SUPPLY CHAIN/ETL/DATA/GOLD/fato_vendas.parquet


In [4]:
# 1) Ler estoque silver (se ainda não tiver lido)
df_estoque_silver = pd.read_parquet(silver_path / "estoque.parquet")
df_estoque_silver['data'] = pd.to_datetime(df_estoque_silver['data'])

# 2) Juntar estoque com calendário para obter id_data
fato_estoque = df_estoque_silver.merge(
    df_calendario_silver[['data', 'id_data']],
    on='data',
    how='left'
)

# 3) Remover 'data' e organizar colunas
fato_estoque = fato_estoque[['id_produto', 'id_loja', 'id_data', 'quantidade_estoque']]

# 4) Checar se há id_data nulo
print("Registros de estoque com id_data nulo:", fato_estoque['id_data'].isna().sum())

# 5) Salvar no GOLD
fato_estoque.to_parquet(gold_path / "fato_estoque.parquet", index=False)

print("fato_estoque salvo em", gold_path / "fato_estoque.parquet")


Registros de estoque com id_data nulo: 0
fato_estoque salvo em /content/drive/MyDrive/PROJETOS/SUPPLY CHAIN/ETL/DATA/GOLD/fato_estoque.parquet


In [5]:
fato_estoque.head()

,id_produto,id_loja,id_data,quantidade_estoque
0,1,1,20220101,425
1,1,1,20220102,421
2,1,1,20220103,419
3,1,1,20220104,414
4,1,1,20220105,407


In [6]:
fato_estoque.to_parquet(gold_path / "fato_estoque.parquet", index=False)
print("fato_estoque salvo em", gold_path / "fato_estoque.parquet")


fato_estoque salvo em /content/drive/MyDrive/PROJETOS/SUPPLY CHAIN/ETL/DATA/GOLD/fato_estoque.parquet
